<a href="https://colab.research.google.com/github/sandhigupta/Sandhi/blob/main/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [21]:
def generateDataSet(num_features, readings):
  data_set = []
  for i in range(num_features):
    thresh = np.random.random()
    if thresh < 0.26:
      data_set.append(list(np.random.normal(20, 2, readings)))
    elif 0.51 > thresh >= 0.26:
      data_set.append(list(np.random.exponential(10, readings)))
    elif 0.76 > thresh >= 0.51:
      data_set.append(list(np.random.random(readings)))
    else:
      data_set.append(list(np.random.beta(5, 4, readings)))
  return (np.array(data_set).transpose(), np.array([0 for i in range(readings)]))

In [22]:
def inject(data, mean, std):
  features = [list(i) for i in data[0]]
  labels = list(data[1])
  for i in range(len(features)):
    if np.random.random() < 0.3333:
      values_to_inject = np.random.normal(mean, std, len(features[0]))
      features[i] = [a - b for a, b in zip(features[i], values_to_inject)]
      labels[i] += 1
  return (np.array(features), np.array(labels))

In [23]:
def dos(data, scale=50):
  features, labels = list(data[0]), list(data[1])
  for i in range(len(features)):
    if np.random.random() < 0.3333:
      to_send = list(np.random.random(len(features[0]))*scale)
      features.insert(i, to_send)
      labels.insert(i, 2)
  return (np.array(features), np.array(labels))

In [24]:
dos_fdia_data = dos(inject(generateDataSet(11, 50), 5, 2))

In [25]:
X_train, X_test, y_train, y_test = train_test_split(dos_fdia_data[0], dos_fdia_data[1], test_size=0.3, random_state=5, shuffle=True)

In [26]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train.ravel())

KNeighborsClassifier()

In [28]:
knn_predictions = knn_model.predict(X_test)
print(f"Accuracy of KNN: {round(accuracy_score(y_test, knn_predictions)*100)}%")

Accuracy of KNN: 95%


In [29]:
print(y_test)
print(knn_predictions)

[2 2 0 1 1 0 0 1 2 1 2 1 0 0 0 1 2 1 1 0 2]
[2 2 0 1 1 0 0 1 2 1 2 0 0 0 0 1 2 1 1 0 2]


In [30]:
df = pd.read_csv(r'LoadMinPower.csv')

df.drop(df.columns[[0,1]], axis=1, inplace=True)
injection = inject((df.to_numpy(), [0 for i in range(len(df))]), 5, 0.5)
features, labels = injection[0], injection[1]

In [31]:
np.savetxt('max_features.csv', features, delimiter=',')
np.savetxt('max_labels.csv', labels, delimiter=',')
max_features = np.array(pd.read_csv('max_features.csv'))
max_labels = np.array(pd.read_csv('max_labels.csv')).astype(int)

In [32]:
features = np.array(pd.read_csv(r'max_features.csv', sep=',', header=0))
labels = np.array(pd.read_csv(r'max_labels.csv', sep=',', header=0)).flatten().astype(int)

In [33]:
load_min_tuple = dos((features, labels))
features, labels = load_min_tuple[0], load_min_tuple[1]

In [34]:
len(labels)

23353

In [35]:
min_preds = knn_model.predict(features)
print(len(min_preds))
print(f"Accuracy of KNN on LoadMinSupply: {round(accuracy_score(labels, min_preds), 2)*100}%")

23353
Accuracy of KNN on LoadMinSupply: 54.0%
